# Non Dyslexic

In [3]:
!sudo apt-get update -y
!sudo apt-get install openjdk-17-jdk -y
!update-alternatives --install /usr/bin/java java /usr/lib/jvm/java-17-openjdk-amd64/bin/java 1
!update-alternatives --set java /usr/lib/jvm/java-17-openjdk-amd64/bin/java



Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,604 kB]
Hit:5 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [8,845 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security/restricted amd64 Packages [4,111 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3,150 kB]
Hit:13 https://ppa.launchpadcontent.net/gr

In [1]:
import zipfile
import os
zip_path = "non_dyslexic.zip"
extract_path = "non_dyslexic_images"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print(f"✅ Unzipped to folder: {extract_path}")


✅ Unzipped to folder: non_dyslexic_images


In [2]:
!pip install textblob language-tool-python eng_to_ipa abydos azure-cognitiveservices-vision-computervision
import os
import io
import time
import pandas as pd
from textblob import TextBlob
import language_tool_python
import eng_to_ipa as ipa
from abydos.phonetic import Soundex, Metaphone, Caverphone, NYSIIS
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials

AZURE_ENDPOINT = "https://dyslexiazd.cognitiveservices.azure.com/"
AZURE_KEY = "5CKFAIsdDlK3y6X1NoWVsUNNq7ZGxNUYCYDc67Ix92ipT5OQa96uJQQJ99BDACYeBjFXJ3w3AAAFACOGlflR"

computervision_client = ComputerVisionClient(
    AZURE_ENDPOINT, CognitiveServicesCredentials(AZURE_KEY)
)
tool = language_tool_python.LanguageTool('en-US')
def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)
    if len(s2) == 0:
        return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

# OCR
def image_to_text(path):
    with open(path, "rb") as image_stream:
        image = image_stream.read()
    read_response = computervision_client.read_in_stream(io.BytesIO(image), raw=True)
    read_operation_location = read_response.headers["Operation-Location"]
    operation_id = read_operation_location.split("/")[-1]

    while True:
        result = computervision_client.get_read_result(operation_id)
        if result.status not in ['notStarted', 'running']:
            break
        time.sleep(1)

    extracted_text = []
    if result.status == OperationStatusCodes.succeeded:
        for page in result.analyze_result.read_results:
            for line in page.lines:
                extracted_text.append(line.text)
    return " ".join(extracted_text)

# Features
def spelling_accuracy(extracted_text):
    spell_corrected = TextBlob(extracted_text).correct()
    return ((len(extracted_text) - levenshtein(extracted_text, spell_corrected)) / (len(extracted_text) + 1)) * 100

def grammatical_accuracy(text):
    matches = tool.check(text)
    total_words = len(text.split())
    errors = len(matches)
    accuracy = (1 - errors / (total_words + 1)) * 100
    return accuracy

def percentage_of_corrections(text, matches=None):
    if matches is None:
        matches = tool.check(text)
    return len(matches) / (len(text.split()) + 1) * 100

def percentage_of_phonetic_accuraccy(extracted_text: str):
    soundex = Soundex()
    metaphone = Metaphone()
    caverphone = Caverphone()
    nysiis = NYSIIS()
    spell_corrected = TextBlob(extracted_text).correct()

    extracted_text_list = extracted_text.split(" ")
    extracted_phonetics_soundex = [soundex.encode(w) for w in extracted_text_list]
    extracted_phonetics_metaphone = [metaphone.encode(w) for w in extracted_text_list]
    extracted_phonetics_caverphone = [caverphone.encode(w) for w in extracted_text_list]
    extracted_phonetics_nysiis = [nysiis.encode(w) for w in extracted_text_list]

    spell_corrected_list = spell_corrected.split(" ")
    spell_corrected_phonetics_soundex = [soundex.encode(w) for w in spell_corrected_list]
    spell_corrected_phonetics_metaphone = [metaphone.encode(w) for w in spell_corrected_list]
    spell_corrected_phonetics_caverphone = [caverphone.encode(w) for w in spell_corrected_list]
    spell_corrected_phonetics_nysiis = [nysiis.encode(w) for w in spell_corrected_list]

    soundex_score = (len(" ".join(extracted_phonetics_soundex)) - levenshtein(" ".join(extracted_phonetics_soundex), " ".join(spell_corrected_phonetics_soundex))) / (len(" ".join(extracted_phonetics_soundex)) + 1)
    metaphone_score = (len(" ".join(extracted_phonetics_metaphone)) - levenshtein(" ".join(extracted_phonetics_metaphone), " ".join(spell_corrected_phonetics_metaphone))) / (len(" ".join(extracted_phonetics_metaphone)) + 1)
    caverphone_score = (len(" ".join(extracted_phonetics_caverphone)) - levenshtein(" ".join(extracted_phonetics_caverphone), " ".join(spell_corrected_phonetics_caverphone))) / (len(" ".join(extracted_phonetics_caverphone)) + 1)
    nysiis_score = (len(" ".join(extracted_phonetics_nysiis)) - levenshtein(" ".join(extracted_phonetics_nysiis), " ".join(spell_corrected_phonetics_nysiis))) / (len(" ".join(extracted_phonetics_nysiis)) + 1)

    return (0.5 * caverphone_score + 0.2 * soundex_score + 0.2 * metaphone_score + 0.1 * nysiis_score) * 100

# Full feature extraction
def get_feature_array(path: str):
    try:
        extracted_text = image_to_text(path)
        features = [
            spelling_accuracy(extracted_text),
            grammatical_accuracy(extracted_text),
            percentage_of_corrections(extracted_text),
            percentage_of_phonetic_accuraccy(extracted_text)
        ]
        return features
    except Exception as e:
        print(f"❌ Error processing {path}: {e}")
        return [None, None, None, None]
    time.sleep(2)

# Process all images and save to CSV
folder = "non_dyslexic_images/non_dyslexic"  # <-- folder where your images are uploaded
label = 0  # Non-dyslexic
data = []

for image_name in os.listdir(folder):
    path = os.path.join(folder, image_name)
    print(f"🖼️ Processing {image_name}...")
    features = get_feature_array(path)
    if None not in features:
        features.append(label)
        data.append(features)
    time.sleep(2)

df = pd.DataFrame(data, columns=[
    "spelling_accuracy",
    "grammatical_accuracy",
    "percentage_of_corrections",
    "percentage_of_phonetic_accuraccy",
    "presence_of_dyslexia"
])

df.to_csv("non_dyslexic_data.csv", index=False)
print("✅ Done. Saved as non_dyslexic_data.csv")


INFO:language_tool_python.download_lt:Unzipping /tmp/tmp8pjw5_0o.zip to /root/.cache/language_tool_python.
INFO:language_tool_python.download_lt:Downloaded https://internal1.languagetool.org/snapshots/LanguageTool-latest-snapshot.zip to /root/.cache/language_tool_python.


🖼️ Processing 43.jpg...
🖼️ Processing 48.jpg...
🖼️ Processing 42.jpg...
🖼️ Processing 20.jpg...
🖼️ Processing 1.jpg...
🖼️ Processing 31.jpg...
🖼️ Processing 27.jpg...
🖼️ Processing 29.jpg...
🖼️ Processing 26.jpg...
🖼️ Processing 44.jpg...
🖼️ Processing 28.jpg...
🖼️ Processing 8.jpg...
🖼️ Processing 45.jpg...
🖼️ Processing 11.jpg...
🖼️ Processing 15.jpg...
🖼️ Processing 35.jpg...
❌ Error processing non_dyslexic_images/non_dyslexic/35.jpg: Operation returned an invalid status code 'Too Many Requests'
🖼️ Processing 32.jpg...
🖼️ Processing 24.jpg...
❌ Error processing non_dyslexic_images/non_dyslexic/24.jpg: Operation returned an invalid status code 'Too Many Requests'
🖼️ Processing 46.jpg...
🖼️ Processing 18.jpg...
🖼️ Processing 5.jpg...
🖼️ Processing 30.jpg...
🖼️ Processing 19.jpg...
🖼️ Processing 10.jpg...
❌ Error processing non_dyslexic_images/non_dyslexic/10.jpg: Operation returned an invalid status code 'Too Many Requests'
🖼️ Processing 39.jpg...
❌ Error processing non_dyslexic_image

# Dyslexic

In [3]:
import zipfile
import os

zip_path = "dyslexic.zip"
extract_path = "dyslexic_images"
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print(f"✅ Unzipped to folder: {extract_path}")


✅ Unzipped to folder: dyslexic_images


In [4]:
!pip install textblob language-tool-python eng_to_ipa abydos azure-cognitiveservices-vision-computervision
import os
import io
import time
import pandas as pd
from textblob import TextBlob
import language_tool_python
import eng_to_ipa as ipa
from abydos.phonetic import Soundex, Metaphone, Caverphone, NYSIIS
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from msrest.authentication import CognitiveServicesCredentials
AZURE_ENDPOINT = "https://dyslexiazd.cognitiveservices.azure.com/"
AZURE_KEY = "5CKFAIsdDlK3y6X1NoWVsUNNq7ZGxNUYCYDc67Ix92ipT5OQa96uJQQJ99BDACYeBjFXJ3w3AAAFACOGlflR"
computervision_client = ComputerVisionClient(
    AZURE_ENDPOINT, CognitiveServicesCredentials(AZURE_KEY)
)
tool = language_tool_python.LanguageTool('en-US')

def levenshtein(s1, s2):
    if len(s1) < len(s2):
        return levenshtein(s2, s1)
    if len(s2) == 0:
        return len(s1)
    previous_row = range(len(s2) + 1)
    for i, c1 in enumerate(s1):
        current_row = [i + 1]
        for j, c2 in enumerate(s2):
            insertions = previous_row[j + 1] + 1
            deletions = current_row[j] + 1
            substitutions = previous_row[j] + (c1 != c2)
            current_row.append(min(insertions, deletions, substitutions))
        previous_row = current_row
    return previous_row[-1]

# OCR
def image_to_text(path):
    with open(path, "rb") as image_stream:
        image = image_stream.read()
    read_response = computervision_client.read_in_stream(io.BytesIO(image), raw=True)
    read_operation_location = read_response.headers["Operation-Location"]
    operation_id = read_operation_location.split("/")[-1]

    while True:
        result = computervision_client.get_read_result(operation_id)
        if result.status not in ['notStarted', 'running']:
            break
        time.sleep(1)

    extracted_text = []
    if result.status == OperationStatusCodes.succeeded:
        for page in result.analyze_result.read_results:
            for line in page.lines:
                extracted_text.append(line.text)
    return " ".join(extracted_text)

# Features
def spelling_accuracy(extracted_text):
    spell_corrected = TextBlob(extracted_text).correct()
    return ((len(extracted_text) - levenshtein(extracted_text, spell_corrected)) / (len(extracted_text) + 1)) * 100

def grammatical_accuracy(text):
    matches = tool.check(text)
    total_words = len(text.split())
    errors = len(matches)
    accuracy = (1 - errors / (total_words + 1)) * 100
    return accuracy

def percentage_of_corrections(text, matches=None):
    if matches is None:
        matches = tool.check(text)
    return len(matches) / (len(text.split()) + 1) * 100

def percentage_of_phonetic_accuraccy(extracted_text: str):
    soundex = Soundex()
    metaphone = Metaphone()
    caverphone = Caverphone()
    nysiis = NYSIIS()
    spell_corrected = TextBlob(extracted_text).correct()

    extracted_text_list = extracted_text.split(" ")
    extracted_phonetics_soundex = [soundex.encode(w) for w in extracted_text_list]
    extracted_phonetics_metaphone = [metaphone.encode(w) for w in extracted_text_list]
    extracted_phonetics_caverphone = [caverphone.encode(w) for w in extracted_text_list]
    extracted_phonetics_nysiis = [nysiis.encode(w) for w in extracted_text_list]

    spell_corrected_list = spell_corrected.split(" ")
    spell_corrected_phonetics_soundex = [soundex.encode(w) for w in spell_corrected_list]
    spell_corrected_phonetics_metaphone = [metaphone.encode(w) for w in spell_corrected_list]
    spell_corrected_phonetics_caverphone = [caverphone.encode(w) for w in spell_corrected_list]
    spell_corrected_phonetics_nysiis = [nysiis.encode(w) for w in spell_corrected_list]

    soundex_score = (len(" ".join(extracted_phonetics_soundex)) - levenshtein(" ".join(extracted_phonetics_soundex), " ".join(spell_corrected_phonetics_soundex))) / (len(" ".join(extracted_phonetics_soundex)) + 1)
    metaphone_score = (len(" ".join(extracted_phonetics_metaphone)) - levenshtein(" ".join(extracted_phonetics_metaphone), " ".join(spell_corrected_phonetics_metaphone))) / (len(" ".join(extracted_phonetics_metaphone)) + 1)
    caverphone_score = (len(" ".join(extracted_phonetics_caverphone)) - levenshtein(" ".join(extracted_phonetics_caverphone), " ".join(spell_corrected_phonetics_caverphone))) / (len(" ".join(extracted_phonetics_caverphone)) + 1)
    nysiis_score = (len(" ".join(extracted_phonetics_nysiis)) - levenshtein(" ".join(extracted_phonetics_nysiis), " ".join(spell_corrected_phonetics_nysiis))) / (len(" ".join(extracted_phonetics_nysiis)) + 1)

    return (0.5 * caverphone_score + 0.2 * soundex_score + 0.2 * metaphone_score + 0.1 * nysiis_score) * 100

# Full feature extraction
def get_feature_array(path: str):
    try:
        extracted_text = image_to_text(path)
        features = [
            spelling_accuracy(extracted_text),
            grammatical_accuracy(extracted_text),
            percentage_of_corrections(extracted_text),
            percentage_of_phonetic_accuraccy(extracted_text)
        ]
        return features
    except Exception as e:
        print(f"❌ Error processing {path}: {e}")
        return [None, None, None, None]


folder = "dyslexic_images/dyslexic"  # <-- folder where dyslexic images are uploaded
label = 1  # Dyslexic
data = []

for image_name in os.listdir(folder):
    path = os.path.join(folder, image_name)
    print(f"🖼️ Processing {image_name}...")
    features = get_feature_array(path)
    if None not in features:
        features.append(label)
        data.append(features)
    time.sleep(3)

df = pd.DataFrame(data, columns=[
    "spelling_accuracy",
    "grammatical_accuracy",
    "percentage_of_corrections",
    "percentage_of_phonetic_accuraccy",
    "presence_of_dyslexia"
])

df.to_csv("dyslexic_data.csv", index=False)
print("✅ Done. Saved as dyslexic_data.csv")


🖼️ Processing 43.jpg...
🖼️ Processing 48.jpg...
🖼️ Processing 42.jpg...
🖼️ Processing 20.jpg...
🖼️ Processing 1.jpg...
🖼️ Processing 31.jpg...
🖼️ Processing 27.jpg...
🖼️ Processing 29.jpg...
🖼️ Processing 26.jpg...
🖼️ Processing 44.jpg...
🖼️ Processing 28.jpg...
🖼️ Processing 8.jpg...
🖼️ Processing 45.jpg...
🖼️ Processing 11.jpg...
🖼️ Processing 15.jpg...
❌ Error processing dyslexic_images/dyslexic/15.jpg: Operation returned an invalid status code 'Too Many Requests'
🖼️ Processing 35.jpg...
🖼️ Processing 32.jpg...
🖼️ Processing 24.jpg...
🖼️ Processing 46.jpg...
❌ Error processing dyslexic_images/dyslexic/46.jpg: Operation returned an invalid status code 'Too Many Requests'
🖼️ Processing 18.jpg...
🖼️ Processing 5.jpg...
🖼️ Processing 30.jpg...
🖼️ Processing 19.jpg...
🖼️ Processing 10.jpg...
🖼️ Processing 39.jpg...
❌ Error processing dyslexic_images/dyslexic/39.jpg: Operation returned an invalid status code 'Too Many Requests'
🖼️ Processing 3.jpg...
🖼️ Processing 37.jpg...
🖼️ Processing 

# Merging

In [5]:
import pandas as pd

df_dyslexic = pd.read_csv("dyslexic_data.csv")
df_non_dyslexic = pd.read_csv("non_dyslexic_data.csv")

df_combined = pd.concat([df_dyslexic, df_non_dyslexic], ignore_index=True)

print("🔍 Combined Dataset Preview:")
print(df_combined.head())
print("\n🚨 Missing Values:")
print(df_combined.isnull().sum())

df_combined.to_csv("combined_dataset.csv", index=False)
print("\n✅ Saved merged dataset as combined_dataset.csv")


🔍 Combined Dataset Preview:
   spelling_accuracy  grammatical_accuracy  percentage_of_corrections  \
0          97.389034             84.507042                  15.492958   
1          90.109890             68.181818                  31.818182   
2          93.886463             83.673469                  16.326531   
3          90.243902             58.181818                  41.818182   
4          90.000000             79.166667                  20.833333   

   percentage_of_phonetic_accuraccy  presence_of_dyslexia  
0                         97.990193                     1  
1                         93.058412                     1  
2                         95.795916                     1  
3                         93.017022                     1  
4                         92.025391                     1  

🚨 Missing Values:
spelling_accuracy                   0
grammatical_accuracy                0
percentage_of_corrections           0
percentage_of_phonetic_accuraccy    0
pr

# Rule Extraction

In [6]:
from sklearn.tree import DecisionTreeClassifier, export_text
from sklearn.model_selection import train_test_split
import pandas as pd

# Load and clean
df = pd.read_csv("combined_dataset.csv")
df_clean = df.dropna()

X = df_clean.drop(columns=["presence_of_dyslexia"])
y = df_clean["presence_of_dyslexia"]

# Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train small interpretable decision tree
tree = DecisionTreeClassifier(max_depth=3, random_state=42)
tree.fit(X_train, y_train)

# Print decision rules
rules = export_text(tree, feature_names=list(X.columns))
print("🌳 Simple interpretable threshold logic:\n")
print(rules)


🌳 Simple interpretable threshold logic:

|--- spelling_accuracy <= 96.25
|   |--- class: 1
|--- spelling_accuracy >  96.25
|   |--- spelling_accuracy <= 97.88
|   |   |--- grammatical_accuracy <= 90.52
|   |   |   |--- class: 0
|   |   |--- grammatical_accuracy >  90.52
|   |   |   |--- class: 1
|   |--- spelling_accuracy >  97.88
|   |   |--- class: 0



In [7]:
# Rule Extraction using Decision Tree
import pandas as pd
from sklearn.tree import DecisionTreeClassifier, export_text

# Load the datasets
df1 = pd.read_csv("non_dyslexic_data.csv")
df2 = pd.read_csv("dyslexic_data.csv")
df = pd.concat([df1, df2], ignore_index=True)

# Features and target
X = df.drop(columns=["presence_of_dyslexia"])
y = df["presence_of_dyslexia"]

# Train a Decision Tree
model = DecisionTreeClassifier(max_depth=3, random_state=42)
model.fit(X, y)

# Display the rules
features = list(X.columns)
tree_rules = export_text(model, feature_names=features)
print("🌳 Simple interpretable threshold logic:\n")
print(tree_rules)


🌳 Simple interpretable threshold logic:

|--- spelling_accuracy <= 96.38
|   |--- class: 1
|--- spelling_accuracy >  96.38
|   |--- spelling_accuracy <= 97.88
|   |   |--- grammatical_accuracy <= 90.52
|   |   |   |--- class: 0
|   |   |--- grammatical_accuracy >  90.52
|   |   |   |--- class: 1
|   |--- spelling_accuracy >  97.88
|   |   |--- class: 0

